In [11]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import os

# Load and preprocess data
data = pd.read_csv("../0_DataPreparation/processed_data.csv")

# Handle missing values in 'Bewoelkung'
data['Bewoelkung'] = data['Bewoelkung'].fillna(data['Bewoelkung'].mean())

# Filter out rows with Umsatz = 0
data = data[data['Umsatz'] != 0]

# Ensure 'Datum' is in datetime format
data['Datum'] = pd.to_datetime(data['Datum'], errors='coerce')

# Define feature columns
feature_columns = [
    'KiWo', 'Is_Weekend', 'Temperature_Category',
    'Windgeschwindigkeit_Beaufort', 'Rain_Status',
    'Bewoelkung','Sommerferien_Flag', 'Christmas_Sales','Warengruppe_1', 'Warengruppe_2',
    'Warengruppe_3', 'Warengruppe_4', 'Warengruppe_5', 'Warengruppe_6'
]

print("Final feature columns used in training:", feature_columns)

# Split data into training and validation sets
training_start_date = '2013-07-01'
training_end_date = '2017-07-31'
validation_start_date = '2017-08-01'
validation_end_date = '2018-07-31'

train_data = data[(data['Datum'] >= training_start_date) & (data['Datum'] <= training_end_date)]
val_data = data[(data['Datum'] >= validation_start_date) & (data['Datum'] <= validation_end_date)]

X_train = train_data[feature_columns].to_numpy()
y_train = train_data['Umsatz'].to_numpy()
X_val = val_data[feature_columns].to_numpy()
y_val = val_data['Umsatz'].to_numpy()

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Build the neural network model
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),  # Input layer
    Dense(32, activation='relu'),                             # Hidden layer
    Dense(16, activation='relu'),                             # Hidden layer
    Dense(1, activation='linear')                             # Output layer for regression
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    verbose=1
)

# Evaluate the model
y_val_pred = model.predict(X_val).flatten()
mse = mean_squared_error(y_val, y_val_pred)
mape = mean_absolute_percentage_error(y_val, y_val_pred)
r2 = r2_score(y_val, y_val_pred)
validation_cost = mse * len(y_val) / 2

print(f"Best R²: {r2:.4f}")
print(f"Validation cost: {validation_cost:.4e}")
print(f"Mean Squared Error (MSE) on validation set: {mse:.4e}")
print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")

# Extract and display model parameters (weights from the first layer)
weights, biases = model.layers[0].get_weights()

print("\nFinal Model Parameters:")
for i, feature in enumerate(feature_columns):
    print(f"Feature: {feature}, Weight: {weights[i][0]:.4f}")
print(f"Intercept (b): {biases[0]:.4f}")

# Save the trained model
model_save_path = "/workspaces/Team_Raum-3_BakerySalesPredictions/3_Model/nn_model.h5"
os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
model.save(model_save_path)
print(f"Model saved to: {model_save_path}")


Final feature columns used in training: ['KiWo', 'Is_Weekend', 'Temperature_Category', 'Windgeschwindigkeit_Beaufort', 'Rain_Status', 'Bewoelkung', 'Sommerferien_Flag', 'Christmas_Sales', 'Warengruppe_1', 'Warengruppe_2', 'Warengruppe_3', 'Warengruppe_4', 'Warengruppe_5', 'Warengruppe_6']
Epoch 1/100


/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


234/234 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 54772.3438 - mae: 182.6550 - val_loss: 5057.1919 - val_mae: 51.0035
Epoch 2/100
234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5349.0464 - mae: 49.5768 - val_loss: 4527.1289 - val_mae: 47.1590
Epoch 3/100
234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5781.5444 - mae: 46.9792 - val_loss: 4324.2271 - val_mae: 45.8127
Epoch 4/100
234/234 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 5079.3936 - mae: 44.6775 - val_loss: 4218.5957 - val_mae: 44.9303
Epoch 5/100
234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4986.5854 - mae: 43.9711 - val_loss: 4358.1753 - val_mae: 46.1485
Epoch 6/100
234/234 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 5163.9087 - mae: 43.9602 - val_loss: 3862.7415 - val_mae: 41.3264
Epoch 7/100
234/234 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4743.4556 - mae: 42.5098 - val_loss: 4108.9321 - val_mae: 43.1819
Epoch 8/100
234/234 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 5129.9370 - mae: 43.4559 - val_loss: 3815.7871 - val_mae: 40

Best R²: 0.7765
Validation cost: 3.4701e+06
Mean Squared Error (MSE) on validation set: 3.7903e+03
Mean Absolute Percentage Error (MAPE): 0.25%

Final Model Parameters:
Feature: KiWo, Weight: -0.2066
Feature: Is_Weekend, Weight: 0.1468
Feature: Temperature_Category, Weight: -0.2169
Feature: Windgeschwindigkeit_Beaufort, Weight: -0.2812
Feature: Rain_Status, Weight: 0.1615
Feature: Bewoelkung, Weight: -0.1535
Feature: Sommerferien_Flag, Weight: 0.1294
Feature: Christmas_Sales, Weight: 0.1653
Feature: Warengruppe_1, Weight: -0.2320
Feature: Warengruppe_2, Weight: 0.2776
Feature: Warengruppe_3, Weight: -0.0323
Feature: Warengruppe_4, Weight: 0.1569
Feature: Warengruppe_5, Weight: 0.2731
Feature: Warengruppe_6, Weight: -0.1921
Intercept (b): 0.0128
Model saved to: /workspaces/Team_Raum-3_BakerySalesPredictions/3_Model/nn_model.h5


In [13]:
import numpy as np
import pandas as pd
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError
from sklearn.preprocessing import StandardScaler

# Paths to the files
processed_data_path = "/workspaces/Team_Raum-3_BakerySalesPredictions/0_DataPreparation/processed_data.csv"
sample_submission_path = "/workspaces/Team_Raum-3_BakerySalesPredictions/0_DataPreparation/sample_submission.csv"
final_submission_path = "/workspaces/Team_Raum-3_BakerySalesPredictions/0_DataPreparation/final_submission.csv"
model_path = "/workspaces/Team_Raum-3_BakerySalesPredictions/3_Model/nn_model.h5"

# Load the processed data and sample submission
processed_data = pd.read_csv(processed_data_path)
sample_submission = pd.read_csv(sample_submission_path)

# Dynamically rebuild feature columns
feature_columns = [
    'KiWo', 'Is_Weekend', 'Temperature_Category',
    'Windgeschwindigkeit_Beaufort', 'Rain_Status',
    'Bewoelkung','Sommerferien_Flag','Christmas_Sales','Warengruppe_1', 'Warengruppe_2',
    'Warengruppe_3', 'Warengruppe_4', 'Warengruppe_5', 'Warengruppe_6'
]

# Extract features for prediction
X_new = processed_data[feature_columns].apply(pd.to_numeric, errors='coerce').fillna(0).to_numpy(dtype=np.float64)

# Load the trained neural network model
model = load_model(
    model_path,
    custom_objects={'mse': MeanSquaredError()}  # Ensure compatibility with saved model
)

# Normalize features using the same scaler used during training
scaler = StandardScaler()
X_new = scaler.fit_transform(X_new)  # Use saved scaler if available

# Predict the output using the trained neural network
y_pred = model.predict(X_new).flatten()

# Add predictions to the processed data DataFrame
processed_data['Predicted_Umsatz'] = y_pred

# Merge predictions with sample submission to ensure matching structure
final_submission = sample_submission[['id']].copy()
final_submission = final_submission.merge(
    processed_data[['ID', 'Predicted_Umsatz']],
    how='left',
    left_on='id',
    right_on='ID'
)

# Drop the redundant 'ID' column and rename 'Predicted_Umsatz' to 'Umsatz'
final_submission.drop(columns=['ID'], inplace=True)
final_submission.rename(columns={'Predicted_Umsatz': 'Umsatz'}, inplace=True)

# Replace null values in the Umsatz column with 0
final_submission['Umsatz'] = final_submission['Umsatz'].fillna(0)

# Save the final submission file
os.makedirs(os.path.dirname(final_submission_path), exist_ok=True)
final_submission.to_csv(final_submission_path, index=False)

print(f"Final submission saved to: {final_submission_path}")


439/439 ━━━━━━━━━━━━━━━━━━━━ 0s 806us/step
Final submission saved to: /workspaces/Team_Raum-3_BakerySalesPredictions/0_DataPreparation/final_submission.csv
